In [5]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import konlpy
from konlpy.tag import Okt
from konlpy.tag import Kkma
import numpy as np
okt = Okt()
kkma = Kkma()

In [6]:

news = pd.read_csv('../김나윤/하이닉스 뉴스 데이터 완성본.csv', encoding='ANSI')
news.head()

,날짜,제목,내용
0,2022.06.24 06:00,삼전·하이닉스 언제 반등?....미 반도체 지수 '촉각',기사내용 요약美필라델피아 반도체 지수 올 들어 33%↓ 하반기 업황도 '흐림'…주가...
1,2022.06.24 06:02,"SK하이닉스, 올해도 반도체 인재 육성 프로그램 실시",[비즈니스 플라자]SK하이닉스가 인재 채용이 필요한 우수 협력사와 취업 준비생을 연...
2,2022.06.24 08:09,"SK하이닉스, 매크로 악재로 메모리 수급 지연…목표가↓-NH",[이데일리 안혜신 기자] NH투자증권은 24일 SK하이닉스(000660)에 대해 ...
3,2022.06.24 08:53,"NH투자증권 “SK하이닉스, 메모리 수급 개선 지연...목표주가 14만원 하향”","글로벌 금리 인상, 유럽 전쟁, 중국 봉쇄 등으로 IT 제품 수요 부진메모리 공급 ..."
4,2022.06.24 18:38,"中, SK하이닉스 키파운드리 인수 승인",SK하이닉스 이천 로고. /연합뉴스 중국이 파운드리(반도체 위탁생산) ...


In [ ]:
# num_data.loc[(num_data['변화량'] > 0), '변화량'] = 1
# num_data.loc[(num_data['변화량'] == 0), '변화량'] = 0.5
# num_data.loc[(num_data['변화량'] < 0), '변화량'] = 0

In [ ]:

# data = pd.read_csv('../김나윤/SK하이닉스 주식 데이터.csv', encoding='ANSI')
# data['변화량'] = data['종가'] - data['시가']
# data.head()

In [ ]:
data['날짜'] = data['날짜'].astype(str)
data['시간'] = data['시간'].astype(str)

data['date'] = data['날짜'] + ' ' + data['시간']

In [ ]:
data['date'] = pd.to_datetime(data['date'], format='%Y%m%d %H%M', errors='raise')

In [ ]:

# data['날짜'] = data['날짜'].astype(str)
# data['시간'] = data['시간'].astype(str)

# data['date'] = data['날짜'] + ' ' + data['시간']
#data['date'] = pd.to_datetime(data['date'])
#data['date'] = data['data'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d %'))
#data['date'] = pd.to_datetime(data['date'], format='%Y%m%d %H%M', errors='raise')
#data.to_csv('../김나윤/SK하이닉스 주식 데이터.csv')
#data['변화량'] = data['종가']-data['시가']

# #data['날짜'] = pd.to_datetime(data['날짜'])
# data['날짜'].min()
data.head()

In [ ]:
# news['날짜'] = pd.to_datetime(news['날짜'])
# data['날짜'] = pd.to_datetime(data['날짜'], '%y-%m-%d %H:%M:%S')

In [ ]:
news.info(), data.info()

In [ ]:
print (news['날짜'].min(), news['날짜'].max())
print (data['날짜'].min(), data['날짜'].max())

In [ ]:
# # f = pd.merge(num_data, news_data)
# print(pd.merge(news_data, num_data, left_on='날짜', right_on='날짜'))

전처리

In [7]:
import re
regex = r'[^\w\s]'
# text 열의 모든 값을 대상으로 정규표현식을 적용하여 특수문자를 제거
news['clean_content'] = news['내용'].apply(lambda x: re.sub(regex, '', str(x)))
news['clean_content'][0]

'기사내용 요약美필라델피아 반도체 지수 올 들어 33 하반기 업황도 흐림주가는 바닥 접근 중 서울뉴시스 김경택 기자  국내 반도체 투톱 삼성전자와 SK하이닉스의 주가가 나란히 신저가 행진을 이어가고 있지만 미국 반도체 기업들도 만만치 않은 낙폭을 기록 중인 것으로 나타났다 경기 침체에 따른 수요 둔화 우려가 글로벌 반도체 기업에 대한 투자심리를 급랭시키고 있는 것으로 보인다 주가 반등의 트리거가 될 반도체 업황 개선 여부에 대해서는 아직 물음표가 붙고 있지만 전문가들 사이에선 최근 주가 하락은 과도하다는 진단이 주를 이루고 있다24일 금융투자업계에 따르면 삼성전자와 SK하이닉스의 주가가 연일 바닥을 뚫고 있는 가운데 미국 대표 반도체 기업들의 주가를 추종하는 필라델피아 반도체 지수 역시 큰 폭의 하락세를 나타내고 있는 것으로 나타났다 필라델피아 반도체 지수는 지난해 말 394617에서 전날22일 261971로 3361 하락했다 이 지수는 글로벌 반도체 업황을 반영하는 지표로 여겨져 시장에서는 반도체주의 조정이 하반기에도 지속될 것이란 우려가 커지고 있다글로벌 반도체 기업의 주가가 동반 부진한 흐름을 보이고 있는 것은 반도체 업황 둔화 우려 때문이다 글로벌 인플레이션 심화 등의 여파로 경기 침체 국면에 진입할 가능성이 높아지고 있고 경기 침체가 가시화될 경우 PC와 스마트폰 노트북 등 IT 제품에 대한 판매량이 줄어 반도체 수요 감소로 이어질 수 있는 것이다실제 반도체 업황의 선행지표로 인식되는 D램 현물 가격도 올 하반기 하락세를 지속할 것으로 전망되고 있다 시장조사업체 트렌드포스는 최근 발표한 보고서를 통해 올해 3분기 D램 가격이 전 분기 대비 최대 8 가량 하락할 것으로 예상했다 재고는 늘고 있는데 수요가 부진하면서 가격이 떨어질 것이라는 예상이다 당초 D램 등 반도체 가격은 지난해 말만 해도 올 상반기 중 바닥을 찍고 하반기에는 반등할 것으로 전망됐다 하지만 글로벌 거시 경제의 환경 변화로 수요 불확실성이 다시 확대됐고 이로 인해 3분기 추가적인 가격 

In [8]:
news

,날짜,제목,내용,clean_content
0,2022.06.24 06:00,삼전·하이닉스 언제 반등?....미 반도체 지수 '촉각',기사내용 요약美필라델피아 반도체 지수 올 들어 33%↓ 하반기 업황도 '흐림'…주가...,기사내용 요약美필라델피아 반도체 지수 올 들어 33 하반기 업황도 흐림주가는 바닥 ...
1,2022.06.24 06:02,"SK하이닉스, 올해도 반도체 인재 육성 프로그램 실시",[비즈니스 플라자]SK하이닉스가 인재 채용이 필요한 우수 협력사와 취업 준비생을 연...,비즈니스 플라자SK하이닉스가 인재 채용이 필요한 우수 협력사와 취업 준비생을 연결해...
2,2022.06.24 08:09,"SK하이닉스, 매크로 악재로 메모리 수급 지연…목표가↓-NH",[이데일리 안혜신 기자] NH투자증권은 24일 SK하이닉스(000660)에 대해 ...,이데일리 안혜신 기자 NH투자증권은 24일 SK하이닉스000660에 대해 매크로 ...
3,2022.06.24 08:53,"NH투자증권 “SK하이닉스, 메모리 수급 개선 지연...목표주가 14만원 하향”","글로벌 금리 인상, 유럽 전쟁, 중국 봉쇄 등으로 IT 제품 수요 부진메모리 공급 ...",글로벌 금리 인상 유럽 전쟁 중국 봉쇄 등으로 IT 제품 수요 부진메모리 공급 증가...
4,2022.06.24 18:38,"中, SK하이닉스 키파운드리 인수 승인",SK하이닉스 이천 로고. /연합뉴스 중국이 파운드리(반도체 위탁생산) ...,SK하이닉스 이천 로고 연합뉴스 중국이 파운드리반도체 위탁생산 업체 키...
...,...,...,...,...
3275,2023.06.23 13:18,"SK하이닉스, 美 IT전시회서 차세대 메모리 솔루션 공개",SK하이닉스는 지난 20일(현지시간)부터 사흘간 미국 라스베이거스에서 열린 IT 전...,SK하이닉스는 지난 20일현지시간부터 사흘간 미국 라스베이거스에서 열린 IT 전시회...
3276,2023.06.23 13:21,"델 이어 HPE까지…SK하이닉스, 美 HPE 전시회서 메모리 솔루션 소개",미국 ICT 기업 HPE 연례행사 HPE 디스커버 참가HBM3·DDR5·CXL·PI...,미국 ICT 기업 HPE 연례행사 HPE 디스커버 참가HBM3DDR5CXLPIM 차...
3277,2023.06.23 15:16,美 고객사들 만난 SK하이닉스…차세대 메모리 대거 공개,지난 20일(현지시간)부터 사흘간 미국 라스베이거스에서 열린 IT 전시회인 'HPE...,지난 20일현지시간부터 사흘간 미국 라스베이거스에서 열린 IT 전시회인 HPE 디스...
3278,2023.06.23 15:54,"삼성·SK하이닉스, 챗GPT용 메모리 시장 선점 경쟁…차세대 HBM 개발 ‘속도전’","삼성전자, 연내 ‘HBM3P’ 개발해 시제품 고객사에 전달”SK하이닉스에 뺏긴 시장...",삼성전자 연내 HBM3P 개발해 시제품 고객사에 전달SK하이닉스에 뺏긴 시장 주도권...


In [9]:
#!pip install konlpy
from konlpy.tag import Kkma        
kkma = Kkma()
from konlpy.tag import Hannanum    
hannanum = Hannanum()
from konlpy.tag import Okt         
t = Okt() 
from konlpy.tag import *
import nltk
import pickle
import copy
from tqdm import tqdm

In [10]:
pos_tag = []

# news_data 데이터프레임의 각 행에 대해 반복
for _, row in tqdm(news.iterrows()):
    # 현재 행의 content 값을 news_text 변수에 할당
    news_text = row['clean_content']
    
    # PosTagging
    tokens_ko = t.pos(news_text)
    pos_tag.append(tokens_ko)

3280it [01:54, 28.59it/s]


In [11]:
#첫번째 행의 Pos Tagging
print(pos_tag[0])

[('기사', 'Noun'), ('내용', 'Noun'), ('요약', 'Noun'), ('美', 'Foreign'), ('필라델피아', 'Noun'), ('반도체', 'Noun'), ('지수', 'Noun'), ('올', 'Verb'), ('들어', 'Verb'), ('33', 'Number'), ('하반기', 'Noun'), ('업', 'Noun'), ('황도', 'Noun'), ('흐림주가는', 'Verb'), ('바닥', 'Noun'), ('접근', 'Noun'), ('중', 'Noun'), ('서울', 'Noun'), ('뉴시스', 'Noun'), ('김경', 'Noun'), ('택', 'Noun'), ('기자', 'Noun'), ('국내', 'Noun'), ('반도체', 'Noun'), ('투톱', 'Noun'), ('삼', 'Modifier'), ('성', 'Modifier'), ('전자', 'Noun'), ('와', 'Josa'), ('SK', 'Alpha'), ('하이닉스', 'Noun'), ('의', 'Josa'), ('주가', 'Noun'), ('가', 'Josa'), ('나란히', 'Adjective'), ('신저', 'Noun'), ('가', 'Josa'), ('행진', 'Noun'), ('을', 'Josa'), ('이', 'Determiner'), ('어가', 'Noun'), ('고', 'Josa'), ('있지만', 'Adjective'), ('미국', 'Noun'), ('반도체', 'Noun'), ('기업', 'Noun'), ('들', 'Suffix'), ('도', 'Josa'), ('만', 'Noun'), ('만치', 'Josa'), ('않은', 'Verb'), ('낙폭', 'Noun'), ('을', 'Josa'), ('기록', 'Noun'), ('중인', 'Noun'), ('것', 'Noun'), ('으로', 'Josa'), ('나타났다', 'Verb'), ('경기', 'Noun'), ('침체', 'Noun'), ('에', 'Jo

In [12]:
normalization_li = []
for pos in pos_tag: 
    in_li = []
    for ele in pos:
        #품사가 조사, 접속사이면 continue
        if ele[1] in ['Josa','Suffix']:
            continue
        in_li.append(ele[0])
    normalization_li.append(in_li)
print(normalization_li[0])

['기사', '내용', '요약', '美', '필라델피아', '반도체', '지수', '올', '들어', '33', '하반기', '업', '황도', '흐림주가는', '바닥', '접근', '중', '서울', '뉴시스', '김경', '택', '기자', '국내', '반도체', '투톱', '삼', '성', '전자', 'SK', '하이닉스', '주가', '나란히', '신저', '행진', '이', '어가', '있지만', '미국', '반도체', '기업', '만', '않은', '낙폭', '기록', '중인', '것', '나타났다', '경기', '침체', '따른', '수요', '둔화', '우려', '글로벌', '반도체', '기업', '대한', '투자', '심리', '급랭', '시키고', '있는', '것', '보인다', '주가', '반등', '트리', '거가', '될', '반도체', '업황', '개선', '여부', '대해', '서는', '아직', '물음표', '붙고', '있지만', '전문가', '사이', '최근', '주가', '하락', '과도', '하다는', '진단', '주', '이루고', '있다', '24일', '금융투자', '업계', '따르면', '삼', '성', '전자', 'SK', '하이닉스', '주가', '연일', '바닥', '뚫고', '있는', '가운데', '미국', '대표', '반도체', '기업', '주가', '추종', '하는', '필라델피아', '반도체', '지수', '역시', '큰', '폭', '하락', '세', '나타내고', '있는', '것', '나타났다', '필라델피아', '반도체', '지수', '지난해', '말', '394617', '전날', '22일', '261971', '로', '3361', '하락', '했다', '이', '지수', '글로벌', '반도체', '업황', '반영', '하는', '지표', '여겨져', '시장', '반도체', '주의', '조정', '하반기', '지속', '될', '것', '우려', '커지고', '있다', '글로벌', '반도체', '기

In [13]:
with open('../김나윤/data/stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.replace('\n','') for x in stopwords] # stopword 파일의 줄바꿈 문자 제거
okt = Okt()

In [14]:
tokens = normalization_li
token_stop = []
for token in tokens:
    in_li = []
    for tok in token:
        if tok not in stopwords:
            in_li.append(tok)
    token_stop.append(in_li)
print(token_stop[0]) 

['기사', '내용', '요약', '美', '필라델피아', '반도체', '지수', '올', '들어', '33', '하반기', '업', '황도', '흐림주가는', '바닥', '접근', '중', '서울', '뉴시스', '김경', '택', '기자', '국내', '반도체', '투톱', '성', 'SK', '하이닉스', '주가', '나란히', '신저', '행진', '어가', '있지만', '미국', '반도체', '기업', '만', '않은', '낙폭', '기록', '중인', '나타났다', '경기', '침체', '따른', '수요', '둔화', '우려', '글로벌', '반도체', '기업', '대한', '투자', '심리', '급랭', '시키고', '있는', '보인다', '주가', '반등', '트리', '거가', '될', '반도체', '업황', '개선', '대해', '서는', '아직', '물음표', '붙고', '있지만', '전문가', '사이', '최근', '주가', '하락', '과도', '하다는', '진단', '주', '이루고', '24일', '금융투자', '업계', '따르면', '성', 'SK', '하이닉스', '주가', '연일', '바닥', '뚫고', '있는', '가운데', '미국', '대표', '반도체', '기업', '주가', '추종', '하는', '필라델피아', '반도체', '지수', '역시', '큰', '폭', '하락', '세', '나타내고', '있는', '나타났다', '필라델피아', '반도체', '지수', '지난해', '말', '394617', '전날', '22일', '261971', '3361', '하락', '했다', '지수', '글로벌', '반도체', '업황', '반영', '하는', '지표', '여겨져', '시장', '반도체', '주의', '조정', '하반기', '지속', '될', '우려', '커지고', '글로벌', '반도체', '기업', '주가', '동반', '부진한', '흐름', '보이', '있는', '반도체', '업황', '둔화', '우려', '때문', '글로

In [15]:
df_li = []
for tokens in token_stop:
    token = ' '.join(tokens)
    df_li.append(token)
df = pd.DataFrame(df_li).rename(columns = {0:'preprocess_context'})
df.head()

,preprocess_context
0,기사 내용 요약 美 필라델피아 반도체 지수 올 들어 33 하반기 업 황도 흐림주가는...
1,비즈니스 플라자 SK 하이닉스 인재 채용 필요한 우수 협력 사와 취업 준비 생 연결...
2,이데일리 안혜신 기자 NH 투자 증권 24일 SK 하이닉스 000660 대해 매크로...
3,글로벌 금리 인상 유럽 전쟁 중국 봉쇄 IT 제품 수요 부진 메모리 공급 증가 원활...
4,SK 하이닉스 이천 로고 연합뉴스 중국 파운드리 반도체 위탁 생산 업체 키 파운드리...


In [16]:
news_data = pd.concat([news , df],axis=1)
#비어있는 5개의 행은 '없음'으로 결측치 처리
news_data['내용'] = news_data['내용'].fillna('없음')
news_data['preprocess_context'] = news_data['preprocess_context'].fillna('없음')
news_data.head()

,날짜,제목,내용,clean_content,preprocess_context
0,2022.06.24 06:00,삼전·하이닉스 언제 반등?....미 반도체 지수 '촉각',기사내용 요약美필라델피아 반도체 지수 올 들어 33%↓ 하반기 업황도 '흐림'…주가...,기사내용 요약美필라델피아 반도체 지수 올 들어 33 하반기 업황도 흐림주가는 바닥 ...,기사 내용 요약 美 필라델피아 반도체 지수 올 들어 33 하반기 업 황도 흐림주가는...
1,2022.06.24 06:02,"SK하이닉스, 올해도 반도체 인재 육성 프로그램 실시",[비즈니스 플라자]SK하이닉스가 인재 채용이 필요한 우수 협력사와 취업 준비생을 연...,비즈니스 플라자SK하이닉스가 인재 채용이 필요한 우수 협력사와 취업 준비생을 연결해...,비즈니스 플라자 SK 하이닉스 인재 채용 필요한 우수 협력 사와 취업 준비 생 연결...
2,2022.06.24 08:09,"SK하이닉스, 매크로 악재로 메모리 수급 지연…목표가↓-NH",[이데일리 안혜신 기자] NH투자증권은 24일 SK하이닉스(000660)에 대해 ...,이데일리 안혜신 기자 NH투자증권은 24일 SK하이닉스000660에 대해 매크로 ...,이데일리 안혜신 기자 NH 투자 증권 24일 SK 하이닉스 000660 대해 매크로...
3,2022.06.24 08:53,"NH투자증권 “SK하이닉스, 메모리 수급 개선 지연...목표주가 14만원 하향”","글로벌 금리 인상, 유럽 전쟁, 중국 봉쇄 등으로 IT 제품 수요 부진메모리 공급 ...",글로벌 금리 인상 유럽 전쟁 중국 봉쇄 등으로 IT 제품 수요 부진메모리 공급 증가...,글로벌 금리 인상 유럽 전쟁 중국 봉쇄 IT 제품 수요 부진 메모리 공급 증가 원활...
4,2022.06.24 18:38,"中, SK하이닉스 키파운드리 인수 승인",SK하이닉스 이천 로고. /연합뉴스 중국이 파운드리(반도체 위탁생산) ...,SK하이닉스 이천 로고 연합뉴스 중국이 파운드리반도체 위탁생산 업체 키...,SK 하이닉스 이천 로고 연합뉴스 중국 파운드리 반도체 위탁 생산 업체 키 파운드리...


In [17]:
news_data.to_csv('하이닉스_preprocess.csv', index=False, encoding='utf-8-sig')

In [19]:
import scipy as sp
import pandas as pd
import numpy as np

from konlpy.tag import Kkma        ; kkma = Kkma()
from konlpy.tag import Hannanum    ; hannanum = Hannanum()
from konlpy.tag import Okt         ; t = Okt()     # 구 트위터
from konlpy.tag import *
import nltk
import pickle

from wordcloud import WordCloud, STOPWORDS
from PIL import Image

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline
import os
import matplotlib.pyplot as plt
import seaborn as sns
#import graphviz
from sklearn.tree import export_graphviz

import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

In [21]:
preprocess = pd.read_csv('하이닉스_preprocess.csv')

In [26]:
preprocess.dropna(axis=0).shape

(3275, 5)

In [22]:
news_text = ''
news_data = preprocess.reset_index(drop=True)
for _, row in news_data.iterrows():
    news_text += row['preprocess_context']

TypeError: can only concatenate str (not "float") to str

In [ ]:
tokens_ko = t.morphs(news_text)
ko = nltk.Text(tokens_ko)

print(len(ko.tokens))          # 토큰 전체 개수
print(len(set(ko.tokens)))     # 토큰 unique 개수
print(ko.vocab().most_common(20))

In [ ]:
# stop_words = ['\n',"'",'…',',','[',']','(',')','"','주','에','코스닥','특징','종목','·','장','코스피','증시','-','적',\
#               '도','기술','분석','마감','‘','`','요약','가','’','의','이','오전','★','은','“','대','”','한','B','로',\
#               '?','3','선','A','오후','는','5','!','"…','상','들','1','만에','제','2','…"','20','일','서','명',"'…",'기',\
#               '···','10','소','등','으로','자','전','률','미','...','50','세','시','안','폭',"…'",'만','9','VI','까지',\
#               '눈','더','e','량','고','인','52','성','띄네','1%','부터','다','감','을','지','4','에도','수','7','것','째',\
#              '체크','기','···','중','계','관련','왜','1억원','총','내','과','젠','또','연','엔','차','굿모닝','할','8','.',\
#              '보다','새','주간','전망','추천','이슈','플러스','사','개월','때','..','임','속','’…','G','나','개','원','에서',\
#              '하는','이유','달','→','권','?…','단독','간','배','30','K','저','와','하','/','1조','6','두','해야','분','형',\
#              '황','공','&','앞두고','보','문','이번','익','X','1억',']"','치','산','를','오','해','S','우리','그','된','준','▶',\
#              '건','재','반','라','10년','초','3분','월','신','p','급','조','줄','경','했다','구','진','이어','올','발','vs','강',\
#              '국','9억','1년','난','판','면','"(','`…','살','아','인데','번','텍','팜','8월','Q','메','2년','점','하고','10월',\
#              'D','비','됐다','채',"]'",'보니','손','확','종','동','팔','40','타','~','9월','2100','30%','땐','말','한다','요',\
#              "',",'스','…`','단','16','길','12','3억','회','될까','호','용','2조','번째','일까','듯','최','하다','이다','되다']

In [ ]:
tokens_ko = [each_word for each_word in tokens_ko
           if each_word not in stop_words]

ko = nltk.Text(tokens_ko)

# 그래프에서 한글 폰트가 깨질 경우 실행
# from matplotlib import font_manager, rc
# font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
# rc('font', family=font_name)

plt.figure(figsize=(15,6))
ko.plot(50)
plt.show()

In [ ]:
data = ko.vocab().most_common(300)
wordcloud = WordCloud(font_path='c:/Windows/Fonts/malgun.ttf',
                     relative_scaling=0.2,
                     background_color='white').generate_from_frequencies(dict(data))

plt.figure(figsize=(8,8))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
#!pip install transformers
import pandas as pd
import numpy as np
import re
import os
from transformers import pipeline